In [3]:
import pyproj
import math
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from utils import ORBSLAMResults, umeyama_alignment, fit_trajectory_nn, predict_trajectory_nn

results = ORBSLAMResults("~/msc/shared_data/orbslam-out-utm-helmert-vabadusepst")

gps_trajectory_wgs = pd.DataFrame([(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                              for kf in results.keyframes[1:]], columns=['lat', 'lon', 'alt'])
slam_trajectory = np.array([(kf.x, kf.y, kf.z) for kf in results.keyframes[1:]])

# Create transformers for WGS84 <-> UTM35N
wgs2utm = pyproj.Transformer.from_crs(4326, 32635)
utm2wgs = pyproj.Transformer.from_crs(32635, 4326)

# Convert GPS trajectory (WGS84) to UTM35N
gps_trajectory_utm = np.array([wgs2utm.transform(kf.gps.lat, kf.gps.lon, kf.gps.alt)
                                   for kf in results.keyframes[1:]])

# Align SLAM trajectory to GPS trajectory
R, t, c  = umeyama_alignment(slam_trajectory.T, gps_trajectory_utm.T, True)
aligned_slam_trajectory_utm = np.array([t + c * R @ p for p in slam_trajectory])

# Convert SLAM trajectory (UTM35N) to WGS84
aligned_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in aligned_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

model = fit_trajectory_nn(aligned_slam_trajectory_utm, gps_trajectory_utm)
fitted_slam_trajectory_utm = predict_trajectory_nn(model, aligned_slam_trajectory_utm)

fitted_slam_trajectory_wgs = pd.DataFrame([utm2wgs.transform(p[0], p[1], p[2]) for p in fitted_slam_trajectory_utm], columns=['lat', 'lon', 'alt'])

fig = go.Figure()
fig.add_trace(
    go.Scattermapbox(lat=gps_trajectory_wgs['lat'], 
                     lon=gps_trajectory_wgs['lon'], 
                     mode='markers+lines',
                     marker=dict(color='blue'),
                     name='GPS'))
fig.add_trace(
    go.Scattermapbox(lat=aligned_slam_trajectory_wgs['lat'], 
                     lon=aligned_slam_trajectory_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='red'),
                     name='SLAM'))
fig.add_trace(
    go.Scattermapbox(lat=fitted_slam_trajectory_wgs['lat'], 
                     lon=fitted_slam_trajectory_wgs['lon'], 
                     mode='markers+lines', 
                     marker=dict(color='forestgreen'),
                     name='fitted SLAM'))
fig.update_geos(projection_type="transverse mercator")
fig.update_layout(mapbox_style="open-street-map",
                  mapbox=dict(center=dict(lat=np.mean(gps_trajectory_wgs['lat']), lon=np.mean(gps_trajectory_wgs['lon'])), zoom=15),
                  margin={"t": 0, "b": 0, "l": 0, "r": 0}, 
                  height=800)
fig.show()

Epoch 1/500
1/1 [==============================] - 0s 228ms/step - loss: 15696402579456.0000
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - loss: 13188054646784.0000
Epoch 3/500
1/1 [==============================] - 0s 4ms/step - loss: 11955567853568.0000
Epoch 4/500
1/1 [==============================] - 0s 5ms/step - loss: 10116133814272.0000
Epoch 5/500
1/1 [==============================] - 0s 4ms/step - loss: 7783569162240.0000
Epoch 6/500
1/1 [==============================] - 0s 10ms/step - loss: 5437573824512.0000
Epoch 7/500
1/1 [==============================] - 0s 5ms/step - loss: 3894571958272.0000
Epoch 8/500
1/1 [==============================] - 0s 5ms/step - loss: 3088191324160.0000
Epoch 9/500
1/1 [==============================] - 0s 5ms/step - loss: 2244006641664.0000
Epoch 10/500
1/1 [==============================] - 0s 4ms/step - loss: 1395156451328.0000
Epoch 11/500
1/1 [==============================] - 0s 4ms/step - loss: 874511990784.0000
Ep

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices